In [1]:
!pip uninstall scikit-learn -y
!pip install -U scikit-learn

Uninstalling scikit-learn-0.24.2:
  Successfully uninstalled scikit-learn-0.24.2
  Using cached https://files.pythonhosted.org/packages/a8/eb/a48f25c967526b66d5f1fa7a984594f0bf0a5afafa94a8c4dbc317744620/scikit_learn-0.24.2-cp37-cp37m-manylinux2010_x86_64.whl


In [2]:
!pip install catboost

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pylab as plt
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10
import gc
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from dateutil import parser
from datetime import datetime
from sklearn.linear_model import LinearRegression
from itertools import product
import torch
from torch import nn, from_numpy
from sklearn.metrics import r2_score,roc_auc_score
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from scipy.sparse import coo_matrix, hstack
from catboost import CatBoostClassifier

In [5]:
X = pd.read_csv(r'/content/drive/MyDrive/classification/train.csv')
X_test = pd.read_csv(r'/content/drive/MyDrive/classification/test.csv')
df_submission = pd.read_csv(r'/content/drive/MyDrive/classification/submission.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (16,26) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
X.drop('machine_id',axis=1,inplace=True)
X_test.drop('machine_id',axis=1,inplace=True)

In [7]:
y = X['is_infected']
X =  X.drop(['is_infected'],axis=1)

In [8]:
X.apply(lambda x: x.unique().shape[0],axis=0)

device_os_version                                  21
device_os_full_name                               313
device_os_build                                    50
device_os_build_revision                          240
device_os_build_lab                               481
device_os_suite                                    11
device_os_platform_name                             4
device_os_platform_subrelease                       9
device_os_architecture                              3
device_os_sku_name                                 22
device_os_install_type_name                         9
device_os_autoupdate_option_name                    6
device_os_is_portable                               2
device_is_protected                                 3
device_is_cloud_defence_enabled                     2
device_is_pua_mode_enabled                          2
device_is_smode_enabled                             3
device_is_smart_screen_enabled                     13
device_is_firewall_enabled  

Датасет содержит множество признаков, имеющих неопределенное влияние на зависимую переменную. Для такой задачи целесообразно использовать градиентный бустинг на решающими деревьями. Т к в задаче множество категориальных признаков используем catboost с удобной обработкой таких признаков. В качестве непрерывных признаков рассмотрим device_system_disk_total_capacity, device_system_volume_total_capacity и device_total_physical_ram, остальные признаки будем полагать категориальными

In [9]:
inds_real = np.where((X.columns=='device_system_disk_total_capacity') | (X.columns=='device_system_volume_total_capacity') | (X.columns=='device_total_physical_ram'))[0]

In [10]:
inds_cat = np.array([i for i in range(X.shape[1]) if i not in inds_real])

In [11]:
X.iloc[:,inds_cat] = X.iloc[:,inds_cat].astype(str)
X_test.iloc[:,inds_cat] = X_test.iloc[:,inds_cat].astype(str)

Заполним пропуски в вещественных переменных медианными значениями, в категориальных переменных создадим новый класс, соответствующий пропуску.

In [12]:
X.iloc[:,inds_real] = X.iloc[:,inds_real].fillna(X.iloc[:,inds_real].median())
X_test.iloc[:,inds_real]=X_test.iloc[:,inds_real].fillna(X_test.iloc[:,inds_real].median())

Оценим сбалансированность классов

In [13]:
y.mean()

0.49978640808137165

Классы сбалансированны

Разобъем датасет на обучение и валидацию. В общем случае лучше использовать кросс валидацию.

In [14]:
X_train,X_val,y_train,y_val = train_test_split(X,y,stratify=y,test_size=0.2)

Зафиксируем параметры, 

In [15]:
model = CatBoostClassifier(iterations=50,learning_rate=1e-1,depth = 10)

In [16]:
model.fit(X_train,y_train,cat_features=inds_cat)

0:	learn: 0.6813522	total: 3.13s	remaining: 2m 33s
1:	learn: 0.6719493	total: 5.98s	remaining: 2m 23s
2:	learn: 0.6642989	total: 8.53s	remaining: 2m 13s
3:	learn: 0.6579508	total: 11.2s	remaining: 2m 8s
4:	learn: 0.6529452	total: 13.7s	remaining: 2m 3s
5:	learn: 0.6488142	total: 16.5s	remaining: 2m
6:	learn: 0.6451000	total: 19s	remaining: 1m 56s
7:	learn: 0.6420899	total: 21.4s	remaining: 1m 52s
8:	learn: 0.6396722	total: 24s	remaining: 1m 49s
9:	learn: 0.6376525	total: 26.5s	remaining: 1m 45s
10:	learn: 0.6358410	total: 29s	remaining: 1m 42s
11:	learn: 0.6342546	total: 31.5s	remaining: 1m 39s
12:	learn: 0.6330446	total: 34s	remaining: 1m 36s
13:	learn: 0.6319189	total: 36.5s	remaining: 1m 33s
14:	learn: 0.6309825	total: 39s	remaining: 1m 31s
15:	learn: 0.6301221	total: 41.6s	remaining: 1m 28s
16:	learn: 0.6292094	total: 44.1s	remaining: 1m 25s
17:	learn: 0.6284558	total: 46.7s	remaining: 1m 23s
18:	learn: 0.6277789	total: 49.2s	remaining: 1m 20s
19:	learn: 0.6271840	total: 51.7s	rema

In [17]:
preds_train = model.predict_proba(X_train)[:,1]
preds_val = model.predict_proba(X_val)[:,1]

In [18]:
print( "ROC-AUC train: {}, ROC-AUC val: {}".format( round(roc_auc_score(y_train,preds_train),3),round(roc_auc_score(y_val,preds_val),3) ) )

ROC-AUC train: 0.72, ROC-AUC val: 0.711


Обучим модель на всех данных и сделаем финальный прогноз.

In [19]:
model = CatBoostClassifier(iterations=50,learning_rate=1e-1,depth = 10)

In [20]:
model.fit(X,y,cat_features=inds_cat)

0:	learn: 0.6810701	total: 3.92s	remaining: 3m 11s
1:	learn: 0.6713434	total: 7.48s	remaining: 2m 59s
2:	learn: 0.6637254	total: 11s	remaining: 2m 52s
3:	learn: 0.6574533	total: 14.2s	remaining: 2m 43s
4:	learn: 0.6526187	total: 17.3s	remaining: 2m 35s
5:	learn: 0.6483453	total: 20.4s	remaining: 2m 29s
6:	learn: 0.6449049	total: 23.5s	remaining: 2m 24s
7:	learn: 0.6421842	total: 26.7s	remaining: 2m 19s
8:	learn: 0.6396521	total: 29.8s	remaining: 2m 15s
9:	learn: 0.6376003	total: 32.9s	remaining: 2m 11s
10:	learn: 0.6359236	total: 36s	remaining: 2m 7s
11:	learn: 0.6343773	total: 39.1s	remaining: 2m 3s
12:	learn: 0.6330241	total: 42.2s	remaining: 2m
13:	learn: 0.6319073	total: 45.4s	remaining: 1m 56s
14:	learn: 0.6308837	total: 48.5s	remaining: 1m 53s
15:	learn: 0.6299518	total: 51.6s	remaining: 1m 49s
16:	learn: 0.6291700	total: 54.7s	remaining: 1m 46s
17:	learn: 0.6283291	total: 57.9s	remaining: 1m 42s
18:	learn: 0.6276510	total: 1m 1s	remaining: 1m 39s
19:	learn: 0.6270797	total: 1m 4

In [21]:
preds = model.predict_proba(X_test)[:,1]
df_submission['is_infected'] = preds

In [23]:
df_submission.to_csv('/content/drive/MyDrive/classification/final_cls.csv')